In [13]:
from pathlib import Path
from collections import deque, defaultdict
from copy import deepcopy

In [14]:
data = Path('Day20.txt').read_text().splitlines()
data = [line.split(' -> ') for line in data]

# Part 1

In [15]:
modules = defaultdict(lambda : {'module_type': '', 'inputs': [], 'outputs': [], 'input_states': [], 'module_state': False})
modules['broadcaster']['inputs'] = ['button']

for module_name, destinations in data:
    if module_name != 'broadcaster':
        module_type, module_name = module_name[:1], module_name[1:]
    else:
        module_type = 'broadcaster'
    
    modules[module_name]['module_type'] = module_type
    modules[module_name]['outputs'] = destinations.split(', ')
    
    for d in destinations.split(', '):
        modules[d]['inputs'].append(module_name)
        modules[d]['input_states'].append(False)

In [16]:
def pulse_output(input_pulse, origin, module, modules):
    module = modules[module]
    mod_type = module['module_type']

    match mod_type:
        case 'broadcaster':
            return input_pulse
        case '%':
            if input_pulse == True:
                return None
            else:
                if module['module_state']:
                    module['module_state'] = False
                    return False
                else:
                    module['module_state'] = True
                    return True
        case '&':
            module['input_states'][module['inputs'].index(origin)] = input_pulse

            if False in module['input_states']:
                return True
            else:
                return False

In [17]:
def circuit_state(modules):
    state = []
    for module, module_props in modules.items():
        if module_props['module_type'] == '%':
            state.append(module_props['module_state'])
        elif module_props['module_type'] == '&':
            state += module_props['input_states']
    
    return tuple(state)

In [20]:
button_presses = 0

high_pulses = 0
low_pulses = 0

modules2 = deepcopy(modules)

for i in range(1000):

    pulses = deque([[False, 'button', 'broadcaster']])

    while pulses:
        input_pulse, origin, module = pulses.popleft()

        if input_pulse:
            high_pulses += 1
        else:
            low_pulses += 1

        output_pulse = pulse_output(input_pulse, origin, module, modules2)

        if output_pulse is None:
            continue

        for dest in modules2[module]['outputs']:
            pulses.append([output_pulse, module, dest])
    
print(high_pulses*low_pulses)

825896364


# Part 2

In [237]:
modules['zg']

{'module_type': '&',
 'inputs': ['vm', 'lm', 'jd', 'fv'],
 'outputs': ['rx'],
 'input_states': [False, False, False, False],
 'module_state': False}

In [239]:
modules2 = deepcopy(modules)

outputs = {module: [] for module in modules2}

for i in range(20000):

    outputs = {module: output_history + [None] for module, output_history in outputs.items()}

    pulses = deque([[False, 'button', 'broadcaster']])

    while pulses:
        input_pulse, origin, module = pulses.popleft()

        output_pulse = pulse_output(input_pulse, origin, module, modules2)

        if output_pulse is None:
            continue
    
        outputs[module][-1] = output_pulse

        if origin in ['vm', 'lm', 'jd', 'fv'] and input_pulse:
            print(origin, i)

        for dest in modules2[module]['outputs']:
            pulses.append([output_pulse, module, dest])

jd 3906
fv 3910
lm 3928
vm 4056
jd 7813
fv 7821
lm 7857
vm 8113
jd 11720
fv 11732
lm 11786
vm 12170
jd 15627
fv 15643
lm 15715
vm 16227
jd 19534
fv 19554
lm 19644


In [240]:
3907*3911*3929*4057

243566897206981